# Recurrent Transfer Mechanism - Simple Demo

A recurrent transfer mechanism is a type of transfer mechanism that extends the functionality of a standard transfer mechanism by allowing to specify the recurrent connections via a matrix.

To make this clear, we will implement a transfer mechanism and a *recurrent* transfer mechanism and compare them.

*Setup and Installation:*

In [ ]:
%%capture
%pip install psyneulink

In [4]:
import psyneulink as pnl

Let's create mechanisms that takes a 3-dimensional input:

In [5]:
transfer = pnl.TransferMechanism(
    input_shapes=3,
)

recurrent_transfer = pnl.RecurrentTransferMechanism(
    input_shapes=3,
)

Let's see how they behave with the same input. For convenience, we define a helper function to execute the mechanisms on multiple inputs and print the result:

In [6]:
def execute_and_print(
        transfer_mech: pnl.TransferMechanism,
        recurrent_mech: pnl.RecurrentTransferMechanism,
        input_sequence: list
):
    """
    Executes a transfer mechanism and a recurrent transfer mechanism on a sequence of inputs.

    This is to demonstrate how the two mechanisms behave with the same inputs. We use sequences
    since the mechanisms are designed to integrate over time through recurrence.
    """
    for idx, input_el in enumerate(input_sequence):
        print('*'* 5 + f' Element {idx} ' + '*'* 5)
        print(f"Input      : {input_el}")
        print(f"Transfer   : {transfer_mech.execute(input_el)}")
        print(f"Transfer(R): {recurrent_mech.execute(input_el)}")
        print()

Let's run the mechanism on a sequence of inputs:

In [7]:
input_sequence = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

execute_and_print(transfer, recurrent_transfer, input_sequence)

***** Element 0 *****
Input      : [1, 0, 0]
Transfer   : [[1. 0. 0.]]
Transfer(R): [[1. 0. 0.]]

***** Element 1 *****
Input      : [0, 1, 0]
Transfer   : [[0. 1. 0.]]
Transfer(R): [[0. 1. 0.]]

***** Element 2 *****
Input      : [0, 0, 1]
Transfer   : [[0. 0. 1.]]
Transfer(R): [[0. 0. 1.]]



We see, that the input is equal to the output. This is because both of the mechanisms by default use no integration and use the identity funtion to transform the output. Let's change the function to see how the mechanisms behave:

In [8]:
transfer = pnl.TransferMechanism(
    input_shapes=3,
    function=pnl.Linear(slope=2, offset=1),
)

recurrent_transfer = pnl.RecurrentTransferMechanism(
    input_shapes=3,
    function=pnl.Linear(slope=2, offset=1)
)

execute_and_print(transfer, recurrent_transfer, input_sequence)

***** Element 0 *****
Input      : [1, 0, 0]
Transfer   : [[3. 1. 1.]]
Transfer(R): [[3. 1. 1.]]

***** Element 1 *****
Input      : [0, 1, 0]
Transfer   : [[1. 3. 1.]]
Transfer(R): [[1. 3. 1.]]

***** Element 2 *****
Input      : [0, 0, 1]
Transfer   : [[1. 1. 3.]]
Transfer(R): [[1. 1. 3.]]



We see the output is now transformed by the linear function. However, we still don't see any integration happening. We can either set integration mode to true (to isolate the integration, we set the function back to indentity):

In [9]:
transfer = pnl.TransferMechanism(
    input_shapes=3,
    integrator_mode=True

)

recurrent_transfer = pnl.RecurrentTransferMechanism(
    input_shapes=3,
    integrator_mode=True
)

execute_and_print(transfer, recurrent_transfer, input_sequence)

***** Element 0 *****
Input      : [1, 0, 0]
Transfer   : [[0.5 0.  0. ]]
Transfer(R): [[0.5 0.  0. ]]

***** Element 1 *****
Input      : [0, 1, 0]
Transfer   : [[0.25 0.5  0.  ]]
Transfer(R): [[0.25 0.5  0.  ]]

***** Element 2 *****
Input      : [0, 0, 1]
Transfer   : [[0.125 0.25  0.5  ]]
Transfer(R): [[0.125 0.25  0.5  ]]



Now, both mechanisms integrate their previous (hidden) state with the current input. The default integration function is an AdaptiveIntegrator with a integration $rate = 0.5$ and implementing this function:

$$
((1-rate) * previous\_value) + (rate * variable)  + noise + offset
$$

with:
- $rate = 0.5$
- $noise = 0$
- $offset = 0$

As for the TransferMechanism, we can either specify the rate, noise and offset in the initialization or define the integrator_function directly:

In [32]:
transfer = pnl.TransferMechanism(
    input_shapes=3,
    integrator_mode=True,
    integration_rate=.25,
)

recurrent_transfer = pnl.RecurrentTransferMechanism(
    input_shapes=3,
    integrator_mode=True,
    integrator_function=pnl.AdaptiveIntegrator(rate=.25)
)

execute_and_print(transfer, recurrent_transfer, input_sequence)

***** Element 0 *****
Input      : [1, 0, 0]
Transfer   : [[0.25 0.   0.  ]]
Transfer(R): [[0.25 0.   0.  ]]

***** Element 1 *****
Input      : [0, 1, 0]
Transfer   : [[0.1875 0.25   0.    ]]
Transfer(R): [[0.1875 0.25   0.    ]]

***** Element 2 *****
Input      : [0, 0, 1]
Transfer   : [[0.140625 0.1875   0.25    ]]
Transfer(R): [[0.140625 0.1875   0.25    ]]



For the RecurrentTransferMechanism, we can also specify the recurrent connection matrix. By default, it is set to the identity matrix (i.e. each node is connected to itself) which is also equivalent to the TransferMechanism. However, we can connect the noes to other nodes by specifying off-diagonal elements in the matrix:

In [18]:
transfer = pnl.TransferMechanism(
    input_shapes=3,
    integrator_mode=True,
)

recurrent_transfer_2 = pnl.RecurrentTransferMechanism(
    input_shapes=3,
    integrator_mode=True,
    matrix=[              # <-- this is ignored
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0]],
)

recurrent_mech_3 = pnl.RecurrentTransferMechanism(default_variable=[[0.0, 0.0, 0.0]],
                                                  auto=1.0,
                                                  hetero=2.0)

comp = pnl.Composition()
comp.add_node(recurrent_transfer_2)

comp.run(inputs={recurrent_transfer_2: input_sequence})

print(comp.results)

#execute_and_print(transfer, recurrent_mech_3, input_sequence)

[[[0.5   0.    0.   ]]

 [[0.25  0.75  0.   ]]

 [[0.125 0.5   0.875]]]


In [ ]:
# /TODO: Finish up when the RecurrentTransferMechanism is "fixed" or misunderstanding is resolved.